In [11]:
def paso_intpoint(u,v):
    # Recorte de paso para el método de puntos interiores
    # para Programación Cuadrática.
    # u es un vector de dimensión p tal que u[i]>0 para todo i
    # v es un vector de dimensión p.
    # Construir un escalar alfa <= 1 tal que
    # u +(alfa)*v >=0.
    # Si v[i] >= 0 para toda i
    # el escalar alfa = 1.
    # En caso contrario existe un índice i tal que v[i] <0.
    #------------------------------------------------
    # Optimización Numérica
    #   ITAM
    #    20 de marzo de 2024
    #-----------------------------------------
    import numpy as np
    p = len(u)
    v_alfa = np.ones(p)
    for i in range(p):
         if(v[i]<0):
             v_alfa[i] = -u[i]/v[i]
             #---Fin de if---
             
     #----------Fin de for -----------

    alfa = np.amin(v_alfa)
    alfa = np.amin([alfa, 1.0])        
    return alfa
    #------- Fin de paso:intpoint.py-----------------
    
    
    
    
    
def myqp_intpoint(Q, A, F,c,b,d):
    # Método de Puntos Interiores para Programación
    # Cuadrática
    # Min (1/2) x.T*Q*x + c.T*x
    # s.a.   A*x = b
    #        F*x >= d
    # Q es simétrica positiva definda de orden n
    # A es matriz mxn tal que rango(A) = m.
    # F matriz pxn.
    # c vector de dimensión n
    # b vector de dimensión m
    # d vector de dimensión 
    #
    # return
    # x vector de dimensión n con el mínimo del problema
    # y vector de dimensión m y es el multiplicador de
    #    Lagrange para: A*x . b = 0
    # mu vector de dimensión p y es el multiplicador de Lagrange
    #    para_ f*x -d >= 0
    # z vector de dimensión p tal que z = F*x-d 
    # iter número de iteraciones que usamos
    #-----------------------------------------------
    # Optimazación Numérica
    #   ITAM
    # 20 de marzo de 2024
    #-----------------------------------------------
    import numpy as np
    import copy
    
    #---------Valores Iniciales---------
    n = len(c)
    m = len(b)
    p = len(d)
    tol = 10**(-5)
    maxiter = 100
    iter = 0
    #-------------Vectores iniciales-----------
    x = np.ones(n)
    y = np.zeros(m)
    mu= np.ones(p)
    z = np.ones(p)
    #--------------------------------
    # lado derecho
    tau = 1/2
    v1 = np.dot(Q,x)+np.dot(A.T, y)-np.dot(F.T,mu) +c
    v2 = np.dot(A,x)-b
    v3 = -np.dot(F,x) +z +d
    v4 = np.multiply(mu,z)

    cnpo = np.concatenate((v1,v2),0)
    cnpo = np.concatenate((cnpo, v3),0)
    cnpo = np.concatenate((cnpo, v4),0)
    norma_cnpo = np.linalg.norm(cnpo)
    #--------------------------------------------
    # Proceso iterativo
    while(norma_cnpo > tol and iter < maxiter):
        cnpo_pert = copy.copy(cnpo)
        cnpo_pert[n+m+p:n+m+p+p]= cnpo_pert[n+m+p:n+m+p+p]-tau
        #---------------------------------------
        #  Matriz de Newton
        dim = n + m + p +p
        M = np.zeros((dim,dim))
        M[0:n, 0:n]  = Q
        M[0:n, n:(n+m)]= A.T
        M[0:n, (n+m):(n+m+p)]=-F.T
        M[n:n+m, 0:n]= A
        M[(n+m):(n+m+p), 0:n]= -F
        M[(n+m):(n+m+p), (n+m+p):(n+m+p+p)]= np.identity(p)
        M[(n+m+p):dim, (n+m):(n+m+p)]= np.diag(z)
        M[(n+m+p):dim, (n+m+p):dim]= np.diag(mu)
        #-------------------------------------
        # Solución del sistema lineal
        dw = np.linalg.solve(M,-cnpo_pert)
        dx = dw[0:n]
        dy = dw[n:n+m]
        dmu =dw[n+m:n+m+p]
        dz =dw[n+m+p:dim]
        #-----------------------------------
        alfa1 = paso_intpoint(mu,dmu)
        alfa2 = paso_intpoint(z,dz)
        alfa = (0.95)*np.amin([alfa1, alfa2, 1.0])
        #---------------------------------
        # Actualizamos los vectores-----
        x =  x  + alfa*dx
        y =  y  + alfa*dy
        mu = mu + alfa*dmu
        z =  z  + alfa*dz
        #--------------------------------
        iter = iter +1
        tau = np.dot(mu,z)/(2*p)
        #------------------------------------
        v1 = np.dot(Q,x)+np.dot(A.T, y)-np.dot(F.T,mu) +c
        v2 = np.dot(A,x)-b
        v3 = -np.dot(F,x) +z +d
        v4 = np.multiply(mu,z)

        cnpo = np.concatenate((v1,v2),0)
        cnpo = np.concatenate((cnpo, v3),0)
        cnpo = np.concatenate((cnpo, v4),0)
        norma_cnpo = np.linalg.norm(cnpo)
        
        print("iter=", iter,"|","||cnpo||=",norma_cnpo)
        #----------------------------------
        if(norma_cnpo <=tol or iter ==maxiter):
           return x,y,mu,z,iter
           break
       
        #  ---- Fin de myqp_intpoint.py----
        #------------------------------------

In [12]:
import numpy as np

# Define the quadratic matrix Q
Q = np.array([[3, -1, 0],
              [-1, 2, -1],
              [0, -1, 1]])

# Define the linear term vector c
c = np.array([-1, -1, -1])

# Define the equality constraint matrix A and vector b
A = np.array([[1, 2, 1]])  # Coefficients of x1 + 2x2 + x3
b = np.array([4])  # The right-hand side of the equality constraint

# Define an inequality constraint: x1 >= -10
F = np.array([[1, 0, 0]])  # Inequality: x1 >= -10
d = np.array([-10])

# Call the function
x_solution, y_solution, mu_solution, z_solution, iter = myqp_intpoint(Q, A, F, c, b, d)

# Print the results
print(f"Solution for x: {x_solution}")
print(f"Multipliers (y): {y_solution}")
print(f"Number of iterations: {iter}")


iter= 1 | ||cnpo||= 8.907173328720036
iter= 2 | ||cnpo||= 7.222233838454953
iter= 3 | ||cnpo||= 4.469420258520553
iter= 4 | ||cnpo||= 0.9339489528673716
iter= 5 | ||cnpo||= 0.046697456216911765
iter= 6 | ||cnpo||= 0.0023349198116000194
iter= 7 | ||cnpo||= 0.00011700471402485701
iter= 8 | ||cnpo||= 7.133997320761301e-06
Solution for x: [0.50000024 1.00000001 1.49999974]
Multipliers (y): [0.49999969]
Number of iterations: 8


In [13]:
Q = np.array([[1, 1, 1],
              [1, 2, 3],
              [1, 3, 4]])

A = np.array([[1, 1, 1]])  # Equality constraint
b = np.array([1])

F = np.array([[1, 0, 0],  # Inequality constraints: x_i >= -100
              [0, 1, 0],
              [0, 0, 1]])

d = np.array([-100, -100, -100])

c = np.array([1, 0, 1])  # Linear term

# Call the myqp_intpoint function
x_solution, y_solution, mu_solution, z_solution, iter = myqp_intpoint(Q, A, F, c, b, d)

# Output the results
print(f"Solution for x: {x_solution}")
print(f"Lagrange multipliers (y): {y_solution}")
print(f"Number of iterations: {iter}")


iter= 1 | ||cnpo||= 171.8620627828236
iter= 2 | ||cnpo||= 168.8002876698927
iter= 3 | ||cnpo||= 162.8158739114611
iter= 4 | ||cnpo||= 151.50999308053304
iter= 5 | ||cnpo||= 130.83827632039834
iter= 6 | ||cnpo||= 94.78335405280492
iter= 7 | ||cnpo||= 41.14266580064798
iter= 8 | ||cnpo||= 2.0571332900323998
iter= 9 | ||cnpo||= 0.1028566645016207
iter= 10 | ||cnpo||= 0.0051428332251469205
iter= 11 | ||cnpo||= 0.0002571416615867045
iter= 12 | ||cnpo||= 1.285708483260431e-05
iter= 13 | ||cnpo||= 6.428639966580216e-07
Solution for x: [ 2.         -2.99999999  2.        ]
Lagrange multipliers (y): [-2.]
Number of iterations: 13


Ejemplo 3

In [14]:
import numpy as np

# Define the problem data
Q = np.array([[2, 0],
              [0, 2]])

A = np.array([[1, 1]])  # Equality constraint x1 + x2 = 1
b = np.array([1])

F = np.array([[1, 0],  # Inequality constraints: x1 >= -10, x2 >= -10
              [0, 1]])

d = np.array([-10, -10])

c = np.array([-4, -6])  # Linear term

# Call the myqp_intpoint function
x_solution, y_solution, mu_solution, z_solution, iter = myqp_intpoint(Q, A, F, c, b, d)

# Output the results
print(f"Solution for x: {x_solution}")
print(f"Lagrange multipliers (y): {y_solution}")
print(f"Number of iterations: {iter}")


iter= 1 | ||cnpo||= 13.922201505724084
iter= 2 | ||cnpo||= 11.289781041885409
iter= 3 | ||cnpo||= 7.15013760716068
iter= 4 | ||cnpo||= 2.069340469174308
iter= 5 | ||cnpo||= 0.10346734252608582
iter= 6 | ||cnpo||= 0.0051750999841437065
iter= 7 | ||cnpo||= 0.00026813195462922627
iter= 8 | ||cnpo||= 3.926141868564052e-05
iter= 9 | ||cnpo||= 1.9472490959135633e-05
iter= 10 | ||cnpo||= 1.021746971660609e-05
iter= 11 | ||cnpo||= 5.364144977143966e-06
Solution for x: [8.72595478e-09 9.99999991e-01]
Lagrange multipliers (y): [4.00000036]
Number of iterations: 11


In [15]:
import os
current_directory = os.getcwd()
current_directory


'/Users/fatimaginebra'